In [ ]:
!source /opt/xilinx/xrt/setup.sh

In [ ]:
!xbutil examine -d 0000:84:00.1 -r thermal

In [ ]:
import pynq
import numpy as np

from pynq import allocate
from pynq import Overlay

from matplotlib import pyplot as plt
from datetime import datetime

In [ ]:
import os
import shutil
parent_dir=os.path.dirname(os.getcwd())


In [ ]:
# Move the overlays to the working directory
overlay_location=parent_dir+'/training/MNIST_Test/HLS_projects/Qkeras_pruned_CNN/hls4ml_alveo_prj/axi_stream_driver.py'
shutil.copyfile(overlay_location,'axi_stream_driver.py')

In [ ]:
def print_img(matrix, title="Label"):
    plt.imshow(matrix, cmap='gray')
    plt.title(title)
    plt.show()

In [ ]:
# Load the xclbin file to the board
from axi_stream_driver import NeuralNetworkOverlay
ol = NeuralNetworkOverlay(xclbin_name="test3.xclbin")

In [ ]:
ol.krnl_rtl_1?

In [ ]:
ol.krnl_rtl_1.register_map

In [ ]:
ol.krnl_rtl_1.signature

### The Inference below is done for the DNN model on the Alveo Board

In [ ]:
# check the data
X_test = np.load("../training/MNIST_Test/Data/X_test.npy")
X_test = np.asarray(X_test, dtype = np.float32)

In [ ]:
i_buff, o_buff = ol.allocate_mem(X_shape=X_test.shape, y_shape=(X_test.shape[0],10), dtype=np.float32)

In [ ]:
y, _, rate = ol.predict(X=X_test, y_shape=(X_test.shape[0],10), input_buffer=i_buff, output_buffer=o_buff, profile=True, debug=False)

In [ ]:
np.save("../training/MNIST_Test/Data/y_alveo_Dense.npy", y)

In [ ]:
ol.free_overlay()

In [ ]:
i_buff, o_buff = ol.allocate_mem(X_shape=X_test.shape, y_shape=(X_test.shape[0],10), dtype=np.float32)

In [ ]:
N_it = 100
rate_v  = []
for i in range(N_it):
    N = int(((i+1)*X_test.shape[0]/N_it))
    in_size = np.linspace(X_test.shape[0]/N_it, X_test.shape[0], N_it, dtype=np.int32)
    i_buff[:N] = X_test[:N]
    y, _, rate = ol.predict(X=X_test[:N], y_shape=(N,10), input_buffer=i_buff[:N], output_buffer=o_buff[:N], profile=True, debug=False)
    rate_v.append(rate)

In [ ]:
plt.plot(in_size, rate_v)

In [ ]:
for i in range(5):
    img_title = ('Predicted label: %d' % np.argmax(np.array(y[i])))
    print_img(X_test[i], img_title)

In [ ]:
y_alveo = y

In [ ]:
%matplotlib inline
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
import plotting

#model_ref = load_model(MODEL_PATH + 'Keras_only/KERAS_check_best_model.h5')
#with tf.device('/cpu:0'):
#    y_ref = model_ref.predict(X_test)

y_test = np.load("../NN_train/MNIST_Test/Data/y_test.npy")
y_test.astype(np.float32)
y_hls = np.load("../NN_train/MNIST_Test/Data/y_hls_Dense.npy")
y_hls.astype(np.float32)

#print("Accuracy baseline:  {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_ref, axis=1))))
print("Accuracy hls4ml: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_hls, axis=1))))
print("Accuracy FPGA: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_alveo, axis=1))))

fig, ax = plt.subplots(figsize=(9, 9))
#_ = plotting.makeRoc(y_test, y_ref, classes)
#plt.gca().set_prop_cycle(None) # reset the colors
_ = plotting.makeRoc(y_test, y_hls, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], linestyle='--')
plt.gca().set_prop_cycle(None) # reset the colors
_ = plotting.makeRoc(y_test, y_alveo   , ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], linestyle=':')

from matplotlib.lines import Line2D
lines = [#Line2D([0], [0], ls='-'),
         Line2D([0], [0], ls='--'),
         Line2D([0], [0], ls=':')]
from matplotlib.legend import Legend
leg = Legend(ax, lines, labels=['hls4ml', 'alveo'],
            loc='lower right', frameon=False)
ax.add_artist(leg)

## Power measures

In [ ]:
from pynq import Device

sensors = Device.active_device.sensors
sensors

In [ ]:
from pynq.pmbus import DataRecorder

recorder = DataRecorder(sensors["12v_aux"].power,
                        sensors["12v_pex"].power,
                        sensors["vccint"].power)

In [ ]:
import pandas as pd

f = recorder.frame

In [ ]:
recorder.record(0.1)

In [ ]:
f.head()

In [ ]:
import plotly.graph_objs as go

layout = {
    'xaxis': {
        'title': 'Time (s)'
    },
    'yaxis': {
        'title': 'Power (W)',
        'rangemode': 'tozero',
        'autorange': True
    }
}

plot = go.FigureWidget(layout=layout)
plot

In [ ]:
def update_data(frame, start, end, plot):
    ranged = frame[start:end]
    average_ranged = frame[start-pd.tseries.offsets.Second(5):end]
    rolling = (average_ranged['12v_aux_power'] + average_ranged['12v_pex_power']).rolling(
        pd.tseries.offsets.Second(5)
    ).mean()[ranged.index]
    powers = pd.DataFrame(index=ranged.index)
    powers['board_power'] = ranged['12v_aux_power'] + ranged['12v_pex_power']
    powers['rolling'] = rolling
    data = [
        go.Scatter(x=powers.index, y=powers['board_power'], name="Board Power"),
        go.Scatter(x=powers.index, y=powers['rolling'], name="5 Second Avg")
    ]
    plot.update(data=data)


In [ ]:
import threading
import time

do_update = True

def thread_func():
    while do_update:
        now = pd.Timestamp.fromtimestamp(time.time())
        past = now - pd.tseries.offsets.Second(60)
        update_data(recorder.frame, past, now, plot)
        time.sleep(0.5)

from threading import Thread
t = Thread(target=thread_func)
t.start()

In [ ]:
do_update = False
t.join()
recorder.stop()